In [1]:
import deep_orderbook as dob

FOLDER data/L2


In [2]:
dir(dob)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'recorder',
 'replay']

In [4]:
receive = dob.recorder.Receiver(['ETHBTC'])

In [8]:
receive.stoprestart(dorestart=False)